# IMPORTANDO BIBLIOTECAS

In [14]:
import pandas as pd
import joblib
import os
import time
from pathlib import Path
from dotenv import load_dotenv
from sqlalchemy import create_engine
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

pd.set_option('display.max_columns', None)
print("Bibliotecas carregadas!")

Bibliotecas carregadas!


# Configuração, variáveis de ambiente e conexão com o banco de dados

In [16]:
BASE_DIR = Path.cwd()
PATH_RATINGS = BASE_DIR / 'ratings.csv'
PATH_LINKS = BASE_DIR / 'links.csv'
ARTEFATO = 'modelo_colaborativo.pkl'

user = os.getenv("DB_USER")
password = os.getenv("DB_PASS")
host = "localhost"
port = "5432"
db_name = os.getenv("DB_NAME")

env_path = BASE_DIR.parent / '.env'
load_dotenv(dotenv_path=env_path)
DB_URL = f"postgresql://{user}:{password}@{host}:{port}/{db_name}"

try:
    engine = create_engine(DB_URL)
    print("Conexão com o banco de dados criada com sucesso!")
except Exception as e:
    print(f"Erro ao conectar com o banco: {e}")

print(f"Artefato será salvo em: {ARTEFATO}")

Conexão com o banco de dados criada com sucesso!
Artefato será salvo em: modelo_colaborativo.pkl


# Carregando dados locais e do movielens

### Aqui fazemos o carregamento do MovieLens e do PostgreSQL. Aplicando o deslocamento (offset) de +1.000.000 nos IDs do MovieLens para evitar conflito com os IDs do banco.

In [9]:
try:
    print("INICIANDO CARGA DE DADOS")
    start_time = time.time()

    query_filmes = "SELECT tmdb_id FROM filmes"
    df_filmes_db = pd.read_sql(query_filmes, engine)
    
    ids_filmes_validos = set(df_filmes_db['tmdb_id'].unique())
    print(f"Total de filmes no catálogo: {len(ids_filmes_validos)}")
    
    print("Carregando MovieLens...")
    df_ml_ratings = pd.read_csv(PATH_RATINGS)
    df_ml_links = pd.read_csv(PATH_LINKS)

    # Limpeza básica e Merge para pegar o tmdbId
    df_ml_links = df_ml_links.dropna(subset=['tmdbId'])
    df_ml_links['tmdbId'] = df_ml_links['tmdbId'].astype(int)
    df_ml_full = df_ml_ratings.merge(df_ml_links, on='movieId')
    df_ml_filtered = df_ml_full[df_ml_full['tmdbId'].isin(ids_filmes_validos)].copy()
    
    df_ml_final = df_ml_filtered[['userId', 'tmdbId', 'rating']].copy()
    df_ml_final.columns = ['id_usuario', 'tmdb_id', 'nota']
    
    # Offset
    df_ml_final['id_usuario'] = df_ml_final['id_usuario'] + 1000000 
    print(f"MovieLens carregado: {len(df_ml_final)} avaliações.")
    print(f"(Removidas {len(df_ml_full) - len(df_ml_final)} avaliações inúteis)")

    # Postgresql
    print("Carregando dados locais do BD")
    query_avaliacoes = """
        SELECT a.id_usuario, a.id_filme AS tmdb_id, a.nota
        FROM avaliacoes a
    """
    df_avaliacoes = pd.read_sql(query_avaliacoes, engine)
    print(f"Avaliações carregadas: {len(df_avaliacoes)}")
    
    print("Carregando Favoritos do BD")
    query_favoritos = """
        SELECT fav.id_usuario, fav.id_filme AS tmdb_id
        FROM favoritos fav
    """
    df_favoritos = pd.read_sql(query_favoritos, engine)

    if not df_favoritos.empty:
        df_favoritos['nota'] = 5.0 
        print(f"{len(df_favoritos)} favoritos transformados em nota 5.0.")
    else:
        df_favoritos = pd.DataFrame(columns=['id_usuario', 'tmdb_id', 'nota'])

    print(f"Carga de dados concluída em {time.time() - start_time:.2f}s")

except Exception as e:
    print("ERRO DURANTE O CARREGAMENTO DOS DADOS")
    print(e)

INICIANDO CARGA DE DADOS
Total de filmes no catálogo: 9119
Carregando MovieLens...
MovieLens carregado: 51866 avaliações.
(Removidas 48957 avaliações inúteis)
Carregando dados locais do BD
Avaliações carregadas: 61
Carregando Favoritos do BD
21 favoritos transformados em nota 5.0.
Carga de dados concluída em 0.08s


# Processamento e junção dos dois datasets

### União dos dois datasets em um único DataFrame para o treinamento.

In [11]:
print("Fundindo os datasets...")
df_local = pd.concat([df_avaliacoes, df_favoritos], ignore_index=True)
df_local = df_local.drop_duplicates(subset=['id_usuario', 'tmdb_id'], keep='first')
df_total = pd.concat([df_ml_final, df_local], ignore_index=True)

print(f"Total de avaliações para treino: {len(df_total)}")
print("Amostra dos dados:")
display(df_total.head(3))

Fundindo os datasets...
Total de avaliações para treino: 51927
Amostra dos dados:


,id_usuario,tmdb_id,nota
0,1000001,862,4.0
1,1000001,15602,4.0
2,1000001,949,4.0


# TREINAMENTO DO MODELO

### Configuração do Reader (escala de notas) e treinamento do algoritmo SVD. Incluí validação cruzada para visualizar a performance (RMSE) antes de treinar o final.

In [12]:
print("INICIANDO TREINAMENTO SVD")
start_train = time.time()

reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(df_total[['id_usuario', 'tmdb_id', 'nota']], reader)

# validação cruzada
print("Executando validação cruzada rápida (3-folds)...")
algo_test = SVD(n_factors=100, n_epochs=20, lr_all=0.005, reg_all=0.02, random_state=42)
cv_results = cross_validate(algo_test, data, measures=['RMSE'], cv=3, verbose=False)
print(f"RMSE Médio estimado: {cv_results['test_rmse'].mean():.4f}")

# Treinamento com todos os dados
print("Treinando modelo com dataset completo...")
trainset = data.build_full_trainset()
modelo_final = algo_test
modelo_final.fit(trainset)

print(f"Modelo treinado em {time.time() - start_train:.2f}s")

INICIANDO TREINAMENTO SVD
Executando validação cruzada rápida (3-folds)...
RMSE Médio estimado: 0.8569
Treinando modelo com dataset completo...
Modelo treinado em 1.54s


# SALVANDO ARTEFATOS

In [13]:
print('Salvando artefato do modelo...')

metadata = {
    "version": "1.4.0",
    "date": time.strftime("%Y-%m-%d %H:%M:%S"),
    "algorithm": "SVD",
    "n_ratings": len(df_total),
    "model": modelo_final
}

joblib.dump(metadata, ARTEFATO)

print(f"Modelo salvo em '{ARTEFATO}' com sucesso.")
print("TREINAMENTO CONCLUÍDO!")

Salvando artefato do modelo...
Modelo salvo em 'modelo_colaborativo.pkl' com sucesso.
TREINAMENTO CONCLUÍDO!
